In [1]:
using LinearAlgebra
using Plots
import StatsBase
include("src/Trajectories.jl")
using .QuTaM

In [2]:
sys = QuTaM.rf_sys
params = QuTaM.rf_params

SimulParameters(psi0=ComplexF64[0.0 + 0.0im, 1.0 + 0.0im]
nsamples=10000
seed=10
ntraj=1000)
multiplier=10.0
tf=5.0
dt=0.005
eps=0.001)

In [3]:
traj = QuTaM.sample_single_trajectory(sys, params, 10)

11-element Vector{Main.QuTaM.DetectionClick}:
 Main.QuTaM.DetectionClick(0.115011501150115, 1)
 Main.QuTaM.DetectionClick(0.115011501150115, 2)
 Main.QuTaM.DetectionClick(0.2000200020002, 1)
 Main.QuTaM.DetectionClick(1.0251025102510252, 2)
 Main.QuTaM.DetectionClick(0.39003900390039004, 1)
 Main.QuTaM.DetectionClick(0.7350735073507351, 2)
 Main.QuTaM.DetectionClick(0.020002000200020003, 1)
 Main.QuTaM.DetectionClick(1.1951195119511953, 2)
 Main.QuTaM.DetectionClick(0.58005800580058, 1)
 Main.QuTaM.DetectionClick(0.22002200220022, 2)
 Main.QuTaM.DetectionClick(0.075007500750075, 1)

In [4]:
cumsum([click.time for click in traj ])

11-element Vector{Float64}:
 0.115011501150115
 0.23002300230023
 0.43004300430043
 1.4551455145514551
 1.8451845184518452
 2.5802580258025802
 2.6002600260026
 3.7953795379537953
 4.3754375437543755
 4.595459545954595
 4.67046704670467

In [ ]:
t_given = collect(LinRange(0,1, 10))

In [27]:
#realmin(Float64)
eps(Float64)

2.220446049250313e-16

In [24]:
typeof(0.1)

Float64

In [ ]:
type

In [5]:
t_given = collect(LinRange(0,1, 10))

10-element Vector{Float64}:
 0.0
 0.1111111111111111
 0.2222222222222222
 0.3333333333333333
 0.4444444444444444
 0.5555555555555556
 0.6666666666666666
 0.7777777777777778
 0.8888888888888888
 1.0

In [6]:
QuTaM.evaluate_at_t(t_given, traj, sys, params.psi0)

10-element Vector{Vector{ComplexF64}}:
 [0.0 + 0.0im, 1.0 + 0.0im]
 [0.0 + 0.0im, 1.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [0.0 + 0.0im, 1.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]
 [1.0 + 0.0im, 0.0 + 0.0im]